In [1]:
import pdfplumber
import markdownify
from markdown_table import markdown_table

def pdf_to_markdown(pdf_path):
    markdown_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                markdown_text += markdownify.markdownify(text) + "\n\n"
            tables = page.extract_tables()
            for table in tables:
                markdown_text += markdown_table([dict(zip(table[0], row)) for row in table[1:]]).get_markdown() + "\n\n"
    return markdown_text

ImportError: cannot import name 'markdown_table' from 'markdown_table' (/Users/anand.thirwani/Desktop/Blogs/B5_Agents_Fin_Analysis/Agentic_AI_Team/venv/lib/python3.10/site-packages/markdown_table/__init__.py)

In [ ]:

pdf_path = "your_file.pdf"
md_content = pdf_to_markdown(pdf_path)
print(md_content)

In [ ]:
# load

In [ ]:
import glob
from IPython.display import Markdown
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader


In [ ]:
pdf_paths = glob.glob('./data/Apple_FY25_Q1_Consolidated_Financial_Statements.pdf')

In [ ]:
# all_documents = []
# # Load and extract text from all PDFs
# for pdf_path in pdf_paths:
#     loader = PyPDFLoader(file_path=pdf_path)
#     documents = loader.load()
#     all_documents.extend(documents)

In [ ]:
# Markdown(all_documents[8].page_content)

In [ ]:
all_documents = []
# Load and extract text from all PDFs
for pdf_path in pdf_paths:
    loader = PDFPlumberLoader(file_path=pdf_path)
    documents = loader.load()
    all_documents.extend(documents)

In [ ]:
Markdown(all_documents[2].page_content)

In [ ]:
import glob
from utils import *
from IPython.display import Markdown

In [ ]:
pdf_paths = glob.glob('./data/*')
vs1 = create_vector_store(pdf_paths)

In [ ]:
manager1 = create_agentic_team(vs1, web_search_required = False)

In [ ]:
# ans = manager1.run("What was YoY revenue growth rate for Apple in quarter 4 ?")
ans = manager1.run("By how much hav NVIDIAs assets increased in Jan 2025 compared to last year ?")

In [ ]:
Markdown(ans)

In [ ]:
# Vector Store

In [ ]:
all_documents = []
# Load and extract text from all PDFs
for pdf_path in pdf_paths:
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.load()
    all_documents.extend(documents)

# Split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator="\n")
split_documents = text_splitter.split_documents(all_documents)

# Generate embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Create FAISS vector store
vectorstore = FAISS.from_documents(split_documents, embeddings)

return vectorstore